In [ ]:
!pip install torch transformers
!pip install datasets


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig, TrainingArguments,Trainer,BertModel
from datasets import load_dataset, load_metric
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from datasets import load_dataset, load_metric

In [ ]:
config = BertConfig.from_pretrained("bert-base-uncased")


In [ ]:
config.num_hidden_layers = 6  # Set the desired number of hidden layers


In [ ]:
model = BertModel(config)


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("glue", "sst2")
metric = load_metric("glue", "sst2")


In [ ]:

import torch
from torch.utils.data import DataLoader
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from transformers import BertTokenizer
from datasets import load_dataset, Dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dataset = load_dataset('glue', 'sst2')

subset_dataset = dataset['train'][:1000] 

train_dataset = Dataset.from_dict(subset_dataset)
train_dataset = train_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

test_dataset = dataset['validation'].map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

train_dataset = dataset["train"]
eval_dataset = dataset["validation"]


config = BertConfig.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create the model architecture for sequence classification
model = BertForSequenceClassification(config)


train_encodings = tokenizer(train_dataset["sentence"], truncation=True, padding=True)
train_labels = train_dataset["label"]

# Prepare your training DataLoader
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_labels)
)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Fine-tune the model
model.train()
for epoch in range(3): 
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Avg Loss: {total_loss/len(train_dataloader):.4f}")


eval_encodings = tokenizer(eval_dataset["sentence"], truncation=True, padding=True)
eval_labels = eval_dataset["label"]

eval_dataset = torch.utils.data.TensorDataset(
    torch.tensor(eval_encodings["input_ids"]),
    torch.tensor(eval_encodings["attention_mask"]),
    torch.tensor(eval_labels)
)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in eval_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print("Accuracy: {:.2%}".format(accuracy))


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Epoch 1 - Avg Loss: 0.4697
Epoch 2 - Avg Loss: 0.2814
Epoch 3 - Avg Loss: 0.2260
Accuracy: 78.33%


In [ ]:
"""
train_dataloader = DataLoader(
    tokenized_datasets["train"], sampler=RandomSampler(tokenized_datasets["train"]), batch_size=8
)
validation_dataloader = DataLoader(
    tokenized_datasets["validation"], sampler=SequentialSampler(tokenized_datasets["validation"]), batch_size=8
)
"""

'\ntrain_dataloader = DataLoader(\n    tokenized_datasets["train"], sampler=RandomSampler(tokenized_datasets["train"]), batch_size=8\n)\nvalidation_dataloader = DataLoader(\n    tokenized_datasets["validation"], sampler=SequentialSampler(tokenized_datasets["validation"]), batch_size=8\n)\n'

In [ ]:
config = BertConfig.from_pretrained("bert-base-uncased")
#config.num_attention_heads = 6
config.num_hidden_layers = 6  # Set the desired number of hidden layers

config.attention_probs_dropout_prob = 0.1
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)
model = model.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.7.attention.self.value.bias', 'bert.encoder.layer.9.output.dense.bias', 'bert.encoder.layer.9.output.dense.weight', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.11.output.dense.weight', 'bert.encoder.layer.7.intermediate.dense.bias', 'bert.encoder.layer.11.attention.output.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'bert.encoder.layer.9.intermediate.dense.bias', 'cls.predictions.transform.dense.bias', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.8.attention.self.query.weight', 'bert.encoder.layer.6.output.dense.bias', 'bert.encoder.layer.6.attention.output.LayerNorm.bias', 'bert.encoder.layer.7.attention.self.key.weight', 'bert.encoder.layer.10.output.dense.weight', 'bert.encoder.layer.9.attention.output.dense.weight', 'bert.encoder.layer.7.attention.self.query.weight', 'bert.enco

In [ ]:
config = model.config

In [ ]:
print(config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.29.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [ ]:
'''
epochs = 3
total_steps = len(train_dataloader) * epochs

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)
'''

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
from transformers import BertConfig, BertModel

config = BertConfig.from_pretrained("bert-base-uncased")
config.num_hidden_layers = 6  # Set the desired number of hidden layers

model = BertModel(config)

# Print the selected layers
selected_layers = model.encoder.layer[:6]
for i, layer in enumerate(selected_layers):
    print(f"Layer {i+1}: {layer}")


Layer 1: BertLayer(
  (attention): BertAttention(
    (self): BertSelfAttention(
      (query): Linear(in_features=768, out_features=768, bias=True)
      (key): Linear(in_features=768, out_features=768, bias=True)
      (value): Linear(in_features=768, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (output): BertSelfOutput(
      (dense): Linear(in_features=768, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (intermediate): BertIntermediate(
    (dense): Linear(in_features=768, out_features=3072, bias=True)
    (intermediate_act_fn): GELUActivation()
  )
  (output): BertOutput(
    (dense): Linear(in_features=3072, out_features=768, bias=True)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
)
Layer 2: BertLayer(
  (attention): BertAttention(
    (self): BertSelfAt

# **Another Model**


```
 6 Head and 6 layers
```



In [ ]:
# Modify the configuration
config = BertConfig.from_pretrained("bert-base-uncased")
config.num_hidden_layers = 6  
config.num_attention_heads = 6  

model = BertModel(config)

In [ ]:

import torch
from torch.utils.data import DataLoader
from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
from datasets import load_dataset
from transformers import BertTokenizer
from datasets import load_dataset, Dataset

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
dataset = load_dataset('glue', 'sst2')

subset_dataset = dataset['train'][:1000] 

train_dataset = Dataset.from_dict(subset_dataset)
train_dataset = train_dataset.map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

test_dataset = dataset['validation'].map(lambda examples: {
    'input_ids': tokenizer(examples['sentence'], padding='max_length', truncation=True)['input_ids'],
    'labels': examples['label']
}, batched=True)

train_dataset = dataset["train"]
eval_dataset = dataset["validation"]


config = BertConfig.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Create the model architecture for sequence classification
model = BertForSequenceClassification(config)


train_encodings = tokenizer(train_dataset["sentence"], truncation=True, padding=True)
train_labels = train_dataset["label"]

# Prepare your training DataLoader
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings["input_ids"]),
    torch.tensor(train_encodings["attention_mask"]),
    torch.tensor(train_labels)
)
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Fine-tune the model
model.train()
for epoch in range(3): 
    total_loss = 0
    for batch in train_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} - Avg Loss: {total_loss/len(train_dataloader):.4f}")


eval_encodings = tokenizer(eval_dataset["sentence"], truncation=True, padding=True)
eval_labels = eval_dataset["label"]

eval_dataset = torch.utils.data.TensorDataset(
    torch.tensor(eval_encodings["input_ids"]),
    torch.tensor(eval_encodings["attention_mask"]),
    torch.tensor(eval_labels)
)
eval_dataloader = DataLoader(eval_dataset, batch_size=16, shuffle=False)

model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for batch in eval_dataloader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        total_correct += (predicted_labels == labels).sum().item()
        total_samples += labels.size(0)

accuracy = total_correct / total_samples
print("Accuracy: {:.2%}".format(accuracy))


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch 1 - Avg Loss: 0.4698
Epoch 2 - Avg Loss: 0.2831
Epoch 3 - Avg Loss: 0.2321
Accuracy: 80.62%


In [ ]:
"""
import torch
from transformers import BertModel

# Define your custom BERT base model
class CustomBertModel(torch.nn.Module):
    def __init__(self):
        super(CustomBertModel, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = torch.nn.Linear(768, 10)  # Example linear layer

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.fc(pooled_output)
        return logits

# Create an instance of your custom BERT base model
model = CustomBertModel()

# Count the total number of parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters in the custom BERT model: {total_params}")
"""